In [1]:
import os
import sys

import semantic_kernel as sk

In [2]:

kernel = sk.Kernel()

In [3]:
import semantic_kernel.connectors.ai.open_ai as skaoai

In [4]:
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
kernel.add_chat_service("azure_chat_completion_service", skaoai.AzureChatCompletion(deployment,endpoint,api_key=api_key,api_version = "2023-12-01-preview"))

In [5]:
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

In [6]:
base_skills_directory = '../../../plugins'

In [7]:
custom_plugin = kernel.import_native_skill_from_directory(base_skills_directory , "APIPlugin")
writer_plugin = kernel.import_semantic_skill_from_directory(base_skills_directory, "WriterPlugin")
email_plugin = kernel.import_semantic_skill_from_directory(base_skills_directory, "EmailPlugin")
translate_plugin = kernel.import_semantic_skill_from_directory(base_skills_directory, "TranslatePlugin")

In [8]:
ask = """
Check the weather in Guangzhou and generate dressing tips based on the results into spanish and write them as travel weather reminder emails
"""
original_plan = await planner.create_plan_async(ask, kernel)

In [9]:
print(original_plan.generated_plan)

{
    "input": "Guangzhou",
    "subtasks": [
        {"function": "APIPlugin.WeatherFunction", "args": {"city": "Guangzhou"}},
        {"function": "WriterPlugin.Tips", "args": {"input": "Weather in Guangzhou", "language": "Spanish"}},
        {"function": "EmailPlugin.WeatherMail", "args": {"input": "Dressing tips based on weather in Guangzhou"}},
        {"function": "TranslatePlugin.MultiLanguage", "args": {"input": "Dressing tips email", "language": "Spanish"}}
    ]
}


In [10]:
await planner.execute_plan_async(original_plan, kernel)

'Correo electrónico con consejos de vestimenta'